In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import os

In [3]:
from google.colab.patches import cv2_imshow

In [4]:
from collections import defaultdict

In [5]:
def segment_threshold(Path):
  
  img = cv2.imread(Path)
  
  # spliting the images
  a, b, c = cv2.split(img)

  # Gaussian blurring 
  blurred = cv2.GaussianBlur(a, (7, 7), 0)

  # thresholding
  rec,th=cv2.threshold(blurred,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

  # inverting the black and white color
  invert = cv2.bitwise_not(th)

  return invert


In [6]:
def erosion(th,iteration,dims):

  # erosion
  eroded=cv2.erode(th,np.ones(dims),iterations=iteration)
  
  return eroded

In [7]:
def dilation(th,iteration,dims):

  # dilation
  dilated=cv2.dilate(th,np.ones(dims),iterations=iteration)
  return dilated

In [11]:
def get_mask_files(path):
  # creating a map to map file with directory
  dir_map=defaultdict(set)
  
  for (root,dirs,files) in os.walk(path):
    dir=root.split("/")[-1]
    for file in files:
      num=file.split("_")[0]
      dir_map[dir].add(num)

  return dir_map

In [9]:
def change_mask_to_bw(path):

  # changing mask to black and white to measure dice score
  img = cv2.imread(path)
  gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  (thresh, im_bw) = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  
  return im_bw

In [12]:
def dice_score(image_1,image_2):

  mask1=(image_1==255)
  mask2=(image_2==255)
  
  # intersection and union
  intersect = np.sum(mask1*mask2)
  fsum = np.sum(mask1)
  ssum = np.sum(mask2)

  # final dice score
  dice = (2 * intersect ) / (fsum + ssum)
  
  return dice


In [14]:
hash_map=get_mask_files("/content/drive/MyDrive/annotations")

def walk(path):
  tot=0     # total dice score  
  count=0   # number of images

  for (root,dirs,files) in os.walk(path):

    for file in files:
      exists=False
      dir=None
      file_num=file.split(".")[0]
      for i in hash_map:
        if file_num in hash_map[i]:
          exists=True
          dir=i
      if exists:
        image_path=os.path.join(path,dir,file)
        segment=segment_threshold(image_path)
        segment=dilation(segment,1,(3,3))
        segment=erosion(segment,1,(3,3))
        file_mask=str(file_num)+"_mask.png"

        mask_image_path=os.path.join("/content/drive/MyDrive/annotations",dir,file_mask)
        mask_bw=change_mask_to_bw(mask_image_path)


        score=dice_score(mask_bw,segment)
        print("the dice score for", file_mask, "is",score )
        tot+=score
        count+=1

        average_dice_score=tot/count
  return average_dice_score


In [15]:
walk("/content/drive/MyDrive/images")

the dice score for 866_mask.png is 0.9748186872244229
the dice score for 771_mask.png is 0.9805684611736801
the dice score for 628_mask.png is 0.9974113952064854
the dice score for 1612_mask.png is 0.9974548253362954
the dice score for 21_mask.png is 0.9959288577101536
the dice score for 1016_mask.png is 0.9924926669671301
the dice score for 146_mask.png is 0.9938268365543212
the dice score for 1573_mask.png is 0.9949272679610275
the dice score for 68_mask.png is 0.9956283078031565
the dice score for 395_mask.png is 0.9840568052884744
the dice score for 1034_mask.png is 0.9843771141755064
the dice score for 828_mask.png is 0.9907647407853235
the dice score for 529_mask.png is 0.9883866777744003
the dice score for 1074_mask.png is 0.989031342106937
the dice score for 1680_mask.png is 0.9962476571960397
the dice score for 862_mask.png is 0.9704654227800213
the dice score for 291_mask.png is 0.9968755894992594
the dice score for 1643_mask.png is 0.9983727852453302
the dice score for 627_m

0.9871047914680796